In [1]:
from dotenv import load_dotenv
import os
import requests
import base64
from datetime import datetime
import json

def get_Activities():
    url = "https://intervals.icu/api/v1/athlete/0/activities?oldest=2020-12-01&newest=2025-12-31"
    
    load_dotenv('APIKEY2.env')
    username = os.getenv("USERNAME")
    password = os.getenv("PASSWORD")
    
    if not username or not password:
        print("Missing credentials.")
        return None

    credentials = base64.b64encode(f"{username}:{password}".encode()).decode()
    
    headers = {
        "Authorization": f"Basic {credentials}",
        "Accept": "application/json"
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

activities = get_Activities()

if activities:
    today = datetime.now().date()
    total_today_km = 0.0

    for activity in activities:
        try:
            activity_date = datetime.fromisoformat(activity['start_date_local'].replace('Z', '+00:00')).date()
            if activity_date == today and activity.get('distance'):
                total_today_km += activity['distance'] / 1000  # meters to km
        except Exception as e:
            print(f"Error: {e}")

    rode_today = total_today_km > 0

    # Save to data.json
    output = {
        "rode_today": rode_today,
        "distance_km": round(total_today_km, 2)
    }

    with open("data.json", "w") as f:
        json.dump(output, f)

    print("✔️ data.json updated.")
else:
    print("❌ Could not get activities.")

/Users/avomref/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✔️ data.json updated.
